# Анализ результатов А/B-тестирования сайта пекарни

Мы хотим усилить визуальный контент на нашем сайте. Есть идея, что если мы добавим видео производства наших изделий, то пользователь будет принимать решение о покупке охотнее и быстрее, и конверсия в заказ вырастет. 

__Продакт вернул нам такой дизайн эксперимента:__

*1. `Бизнес-проблема:`* мы считаем, что конверсии из визита в покупку в нашей пекарне недостаточно высоки. Мы посмотрели на страницы конкурентов и поняли, что у нас есть возможность увеличить объем продаж, если мы изменим дизайн страницы с каталогом продукции.

*2. `Гипотеза:`* предполагается, что добавление видео с процессом приготовления вкусностей мотивирует к их покупке и позволяет добиться двух эффектов:
   * повышение конверсии из визита в оплату (больше людей захотят купить нашу продукцию)
   * как минимум, не снижение среднего чека покупки, а в идеале — увеличение
    
*3. `Ожидаемые действия пользователей:`* мы ожидаем, что, увидев видео с процессом приготовления вкусностей вместе с описанием продукции, больше людей захотят сделать покупку, а возможно — в среднем будут покупать больше продукции в одном заказе.

*4. `Эксперимент будет считаться успешным, если:`*
   * конверсия из визита в покупку с окном в 7 дней повысится
   * средний чек покупки не снизится


## Что нужно сделать?

Проанализировать данные теста и дать свои комментарии о его успешности. Можно ли выкатывать изменение на всех пользователей?

1. Выбрать окно конверсии
2. Исследовать распределение пользователей по группам по основным сегментам
3. Оценить статистическую значимость в изменении конверсии с помощью критерия хи-квадрат
4. Вычислить мощность теста
5. Использовать бутстрап для оценки влияния теста на средний чек
6. Сделать вывод по результатам проведенного теста

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Задание 1. Выбор окна конверсии на основе исторических данных

**1. Подлючаемся к историческим данным.**

In [ ]:
historical_data = pd.read_csv('investigate_window.csv', parse_dates = [2,3], index_col = 0)

historical_data.head()

In [ ]:
historical_data.dtypes

**2. Вычисляем, какой перцентиль соответствует недельному окну конверсии.**

In [ ]:
np.percentile(historical_data['time_to_order'], 95)

In [ ]:
np.percentile(historical_data['time_to_order'], 90)

In [ ]:
np.percentile(historical_data['time_to_order'], 85)

In [ ]:
# недельное окно конверсии

perc_95 = round(np.percentile(historical_data['time_to_order'], 95), 1)

**3. Строим гистограмму распределения времени между первым заходом на сайт и покупкой.**

In [ ]:
plt.figure(figsize=(16,9))

plt.hist(historical_data['time_to_order'], bins = 40)

plt.axvline(perc_95,
            ymin=0,
            ymax=0.7,
            color='k',
            linestyle='dashed',
            label=f'95-ый перцентиль – {perc_95} дня'
            )

plt.title('Распределение времени между первым заходом на сайт и покупкой')
plt.xlabel('Количество дней')
plt.ylabel('Количество пользователей')

plt.legend()

plt.show()

**4. Делаем вывод об окне конверсии.**

Недельному окну конверсии соответствует 95-й процентиль распределения времени между первым заходом на сайт и покупкой.

## Задание 2. Исследование распределения пользователей по группам по основным сегментам 

**1. Подключаемся к данным A/B теста.**

In [ ]:
data = pd.read_csv('bakery_ab_test.csv', index_col = 0)

data.head()

**2. Создаем функцию plot_segment_distribution, чтобы проверить распределение пользователей по сегментам внутри каждой группы.**

In [ ]:
def plot_segment_distribution(df, segment_columns, test_factor):
    for segment in segment_columns:
        aggregated_data = df.groupby(by = [test_factor, segment])['user_id'].count().reset_index()
        sns.catplot(x = segment, 
                    y = 'user_id', 
                    hue = test_factor, # разный цвет для групп
                    data = aggregated_data, 
                    kind = "bar", 
                    height = 4, # размер графика 
                    aspect = 1.5) # ширина столбца

**3. Строим графики при помощи созданной функции.**

In [ ]:
plot_segment_distribution(data, 
                          ["geo_group", "acquisition_channel", "platform"],
                          "test_group")

**4. Делаем вывод о качестве сплитования.**

В рамках сегментов `geo_group` и `acquisition_channel` группы распределены равномерно, в рамках сегмента `platform` распределение неравномерное.

## Задание 3. Критерий Хи-квадрат для конверсии


Мы не можем исправить данные, но можем проанализировать общие результаты и сравнить их с результатами для пользователей, зашедших с разных платформ, чтобы уменьшить риск принятия ошибочного решения. Поэтому дальше мы будем анализировать параллельно 3 теста:

1. Все пользователи, попавшие в тест
2. Пользователи, заходившие с ПК `platform = 'pc'`
3. Пользователи, заходившие с мобильного `platform = 'mobile'`

**1. Создаем для каждой из 3 групп теста отдельную таблицу с метриками.**

In [ ]:
metrics = data.groupby('test_group', as_index= False)\
    .agg({'user_id': 'count', 'within_window': 'sum', 'purchase_amount': 'mean'}).reset_index()
metrics['conversion'] = round(100 * metrics['within_window'] / metrics['user_id'], 2)
metrics

In [ ]:
metrics_pc = data.loc[data['platform'] == 'pc'].groupby('test_group', as_index= False)\
    .agg({'user_id': 'count', 'within_window': 'sum', 'purchase_amount': 'mean'}).reset_index()
metrics_pc['conversion'] = round(100 * metrics_pc['within_window'] / metrics_pc['user_id'], 2)
metrics_pc

In [ ]:
metrics_mobile = data.loc[data['platform'] == 'mobile'].groupby('test_group', as_index= False)\
    .agg({'user_id': 'count', 'within_window': 'sum', 'purchase_amount': 'mean'}).reset_index()
metrics_mobile['conversion'] = round(100 * metrics_mobile['within_window'] / metrics_mobile['user_id'], 2)
metrics_mobile

**2. Вычисляем p-value для каждой группы с помощью теста хи-квадрат.**

In [ ]:
import statsmodels.stats.proportion as proportion

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(metrics['within_window'], metrics['user_id'])

chi2stat_pc, pval_pc, table_pc = proportion.proportions_chisquare(metrics_pc['within_window'], metrics_pc['user_id'])

chi2stat_mobile, pval_mobile, table_mobile = proportion.proportions_chisquare(metrics_mobile['within_window'], metrics_mobile['user_id'])

**3. Вводим в анализ уровень значимости в 5% с помощью переменной alpha.**

In [ ]:
alpha = 0.05

In [ ]:
pval < alpha

In [ ]:
pval_pc < alpha

In [ ]:
pval_mobile < alpha

**4. Делаем вывод о наличии статистически значимой разницы в каждой группе.**

P-value во всех трех группах меньше заданного значения уровня значимости, поэтому мы принимаем нулевую гипотезу о том, что различий в конверсии нет (при условии достаточной мощности).

## Задание 4. Вычисление мощности теста

**1. Записываем значения конверсии в тестовой и контрольной группе в разные переменные.**

In [ ]:
conversion_control = metrics['conversion'].values[0]/100
conversion_test = metrics['conversion'].values[1]/100

In [ ]:
conversion_control_pc = metrics_pc['conversion'].values[0]/100
conversion_test_pc = metrics_pc['conversion'].values[1]/100

In [ ]:
conversion_control_mobile = metrics_mobile['conversion'].values[0]/100
conversion_test_mobile = metrics_mobile['conversion'].values[1]/100

**2. Создаем переменную nobs для количества наблюдений и записываем в нее значение меньшей группы.**

In [ ]:
nobs = min(metrics['user_id'])

In [ ]:
nobs_pc = min(metrics_pc['user_id'])

In [ ]:
nobs_mobile = min(metrics_mobile['user_id'])

**3. Создаем функцию для того, чтобы посчитать, насколько одна группа эффективнее другой в исследуемой метрике.**

In [ ]:
def chi2_effect_size(p0, p1):
    return np.sqrt(((p0 - p1)**2 / p0))

**4. Вычисляем мощность для каждой группы, для которой делали тест хи-квадрат.**

In [ ]:
import statsmodels.stats.power as power

In [ ]:
chipower = power.GofChisquarePower()

In [ ]:
power = chipower.solve_power(effect_size = chi2_effect_size(conversion_control, conversion_test), # разница коэффициентов конверсии
                             nobs = nobs, # размер выборки
                             alpha = pval, # значение ошибки первого рода
                             power = None) # функция вернет значение, указаное как None
power

In [ ]:
pc_power = chipower.solve_power(effect_size = chi2_effect_size(conversion_control_pc, conversion_test_pc), # разница коэффициентов конверсии
                                nobs = nobs_pc, # размер выборки
                                alpha = pval_pc, # значение ошибки первого рода
                                power = None) # функция вернет значение, указаное как None
pc_power

In [ ]:
mobile_power = chipower.solve_power(effect_size = chi2_effect_size(conversion_control_mobile, conversion_test_mobile), # разница коэффициентов конверсии
                                    nobs = nobs_mobile, # размер выборки
                                    alpha = pval_mobile, # значение ошибки первого рода
                                    power = None) # функция вернет значение, указаное как None
mobile_power

**5. Делаем вывод о величине мощности в каждой группе.**

Мощность по всем пользователям больше 90%, в сегменте `'pc'` около 80%, в сегменте `'mobile'` гораздо ниже 80%, поэтому в случае последнего нельзя сделать достоверные выводы.

## Задание 5. Использование бутстрапа для оценки влияния теста на средний чек

**1. Записываем данные по суммам покупки в тестовой и контрольной группе в разные переменные.**

In [ ]:
test = data[data['test_group'] == 'test']['purchase_amount'].dropna().values
control = data[data['test_group'] == 'control']['purchase_amount'].dropna().values

In [ ]:
test_pc = data[(data['test_group'] == 'test') & (data['platform'] == 'pc')]['purchase_amount'].dropna().values
control_pc = data[(data['test_group'] == 'control') & (data['platform'] == 'pc')]['purchase_amount'].dropna().values

In [ ]:
test_mobile = data[(data['test_group'] == 'test') & (data['platform'] == 'mobile')]['purchase_amount'].dropna().values
control_mobile = data[(data['test_group'] == 'control') & (data['platform'] == 'mobile')]['purchase_amount'].dropna().values

**2. Вызываем функцию bootstrap_ab для каждой из трех групп.**

In [ ]:
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import bootstrapped.compare_functions as bs_compare

In [ ]:
boot = bs.bootstrap_ab(test = test, 
                       ctrl = control,
                       stat_func = bs_stats.mean,
                       compare_func = bs_compare.difference,
                       return_distribution=True)

In [ ]:
boot_pc = bs.bootstrap_ab(test = test_pc, 
                          ctrl = control_pc,
                          stat_func = bs_stats.mean,
                          compare_func = bs_compare.difference,
                          return_distribution=True)

In [ ]:
boot_mobile = bs.bootstrap_ab(test = test_mobile, 
                              ctrl = control_mobile,
                              stat_func = bs_stats.mean,
                              compare_func = bs_compare.difference,
                              return_distribution=True)

**3. Создаем функцию для визуализации результатов bootstrap теста.**

In [ ]:
def plot_bootstrap_test(data, title):

    borders = np.percentile(data, [2.5, 97.5]) # задаем ширину 95%-ного доверительного интервала
    
    ax = sns.histplot(data)
    ax.set(xlabel=None, ylabel=None)
    
    plt.title(title, fontsize=12)
    plt.rcParams["figure.figsize"] = (10, 8)
    
    plt.axvline(0, # рисуем линию на уровне ноля
               ymin=0, 
               ymax=0.5, 
               linestyle='--', 
               color='red')
    
    plt.axvline(borders[0], # рисуем линию в начале доверительного интервала 
               ymin=0, 
               ymax=0.5, 
               linestyle='--', 
               color='grey', 
               label='95%-доверительный интервал')
    
    plt.axvline(borders[1], # рисуем линию в конце доверительного интервала
               ymin=0, 
               ymax=0.5, 
               linestyle='--', 
               color='grey')
    
    plt.legend(loc="upper right")
    
    plt.show()

**4. Визуализируем результат по всем группам при помощи созданной функции.**

In [ ]:
plot_bootstrap_test(data=boot,
                    title='Bootstrap-распределение разниц среднего чека - общее')

In [ ]:
plot_bootstrap_test(data=boot_pc,
                    title='Bootstrap-распределение разниц среднего чека - pc')

In [ ]:
plot_bootstrap_test(data=boot_mobile,
                    title='Bootstrap-распределение разниц среднего чека - mobile')

**5. Делаем вывод о том, есть ли статистически значимое изменение среднего чека в каждой группе.**

Во всех трех случаях ноль не входит в доверительный интервал, значит есть статистически значимое изменение среднего чека. Во всех трех случаях средний чек увеличился.

## Задание 6. Вывод по результатам проведенного теста

Эксперимент нельзя считать успешным, т.к. статистически значимого измененения в конверсии нет. Но при этом есть статистически значимое увеличение среднего чека.

Т.к. есть ошибка в сплитовании (наблюдения неравномерно распределены на тестовую и контрольную группу в сегменте `platform`) и величина мощности недостаточна в тесте по группе `mobile`, эксперимент лучше повторить, исправив эти недостатки.